### db connection

In [1]:
import dash
from dash import Dash, html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from sqlalchemy import create_engine

# Replace with your credentials
username = 'postgres'
password = 'Mission'
host = 'localhost'
port = '5432'       # default PostgreSQL port
database = 'player_dashboard'

engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')


# Test connection
try:
    connection = engine.connect()
    print("✅ Successfully connected to PostgreSQL!")
    connection.close()
except Exception as e:
    print(f"❌ Connection failed: {e}")


✅ Successfully connected to PostgreSQL!


In [8]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
from sqlalchemy import create_engine
from functools import lru_cache

# -------------------- CONFIGURATION -------------------- #

DB_CONFIG = {
    'username': 'postgres',
    'password': 'Mission',
    'host': 'localhost',
    'port': '5432',
    'database': 'player_dashboard'
}

EVENT_TYPES = ["Pass", "Carry", "Dribble", "Shot", "Duel"]

engine = create_engine(
    f"postgresql+psycopg2://{DB_CONFIG['username']}:{DB_CONFIG['password']}@"
    f"{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
)

# -------------------- DATABASE FUNCTIONS -------------------- #

def load_players(engine):
    query = """
        SELECT DISTINCT p.player_id, p.name
        FROM players p
        JOIN events e ON p.player_id = e.player_id
        ORDER BY p.name;
    """
    df = pd.read_sql(query, engine)
    options = [{"label": row.name, "value": row.player_id} for row in df.itertuples()]
    return options

def load_player_matches(player_id, engine):
    query = """
        SELECT DISTINCT
            m.match_id,
            CASE
                WHEN e.team_id = t_home.team_id THEN m.away_team
                ELSE m.home_team
            END AS opponent
        FROM events e
        JOIN matches m ON e.match_id = m.match_id
        JOIN teams t_home ON m.home_team = t_home.team
        JOIN teams t_away ON m.away_team = t_away.team
        WHERE e.player_id = %s
    """
    df = pd.read_sql(query, engine, params=(player_id,))
    df['label'] = 'vs ' + df['opponent']
    return [{'label': row.label, 'value': row.match_id} for row in df.itertuples()]

def fetch_player_events(player_id, engine, match_ids=None):
    base_query = """
        SELECT type, location_x, location_y, shot_outcome
        FROM events
        WHERE player_id = %s
          AND type = ANY(%s)
          AND location_x IS NOT NULL
          AND location_y IS NOT NULL
    """
    if match_ids:
        query = base_query + " AND match_id = ANY(%s)"
        df = pd.read_sql(query, engine, params=(player_id, EVENT_TYPES, list(match_ids)))
    else:
        df = pd.read_sql(base_query, engine, params=(player_id, EVENT_TYPES))

    return df[df['location_x'].between(0, 120) & df['location_y'].between(0, 80)]

# -------------------- HEATMAP FUNCTIONS -------------------- #

def draw_pitch_vertical(fig):
    pitch_shapes = [
        dict(type="rect", x0=0, y0=0, x1=80, y1=120, line=dict(color="black")),
        dict(type="line", x0=0, y0=60, x1=80, y1=60, line=dict(color="black")),
        dict(type="rect", x0=18, y0=0, x1=62, y1=18, line=dict(color="black")),
        dict(type="rect", x0=18, y0=102, x1=62, y1=120, line=dict(color="black")),
        dict(type="rect", x0=30, y0=0, x1=50, y1=6, line=dict(color="black")),
        dict(type="rect", x0=30, y0=114, x1=50, y1=120, line=dict(color="black")),
        dict(
            type="circle",
            x0=40 - 9.15, y0=60 - 9.15,
            x1=40 + 9.15, y1=60 + 9.15,
            line=dict(color="black")
        )
    ]

    fig.update_layout(shapes=pitch_shapes)
    fig.add_trace(go.Scatter(
            x=[40, 40], y=[12, 108],
            mode='markers',
            marker=dict(color="black", size=4),
            hoverinfo='skip',
            showlegend=False
        ))

    return fig

@lru_cache(maxsize=128)
def compute_kde_cached(player_id, match_ids):
    df = fetch_player_events(player_id, engine, match_ids)
    if df.empty:
        return None

    x, y = df['location_y'].values, df['location_x'].values

    kde = gaussian_kde([x, y])
    xi = np.linspace(0, 80, 160)
    yi = np.linspace(0, 120, 240)
    xi_grid, yi_grid = np.meshgrid(xi, yi)
    z = kde(np.vstack([xi_grid.ravel(), yi_grid.ravel()])).reshape(xi_grid.shape)
    z[z < z.max() * 0.05] = np.nan

    return xi, yi, z, x, y, df['type'].values, df['shot_outcome'].fillna("").values

def compute_kde(player_id, match_ids=None):
    match_ids = tuple(match_ids) if match_ids else None
    return compute_kde_cached(player_id, match_ids)

def create_vertical_heatmap(kde_result):
    if kde_result is None:
        return go.Figure().update_layout(title="No event data available")

    xi, yi, z, x, y, types, outcomes = kde_result
    fig = go.Figure()

    fig.add_trace(go.Contour(
        z=z, x=xi, y=yi,
        colorscale=[
            [0.0, 'rgb(255,255,255)'],
            [0.05, 'rgb(255,255,255)'],
            [0.2, '#fdbb84'],
            [0.4, '#fc8d59'],
            [0.6, '#ef6548'],
            [0.8, '#d7301f'],
            [1.0, '#990000']
        ],
        opacity=0.85,
        showscale=False,
        hoverinfo='skip',
        contours=dict(showlines=False),
        showlegend=False
    ))

    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers',
        marker=dict(size=2, color='black', opacity=0.3),
        hoverinfo='skip',
        showlegend=False
    ))

    shot_x_goal = [x[i] for i in range(len(types)) if types[i] == 'Shot' and outcomes[i] == 'Goal']
    shot_y_goal = [y[i] for i in range(len(types)) if types[i] == 'Shot' and outcomes[i] == 'Goal']
    shot_x_miss = [x[i] for i in range(len(types)) if types[i] == 'Shot' and outcomes[i] != 'Goal']
    shot_y_miss = [y[i] for i in range(len(types)) if types[i] == 'Shot' and outcomes[i] != 'Goal']

    if shot_x_miss and shot_y_miss:
        fig.add_trace(go.Scatter(
            x=shot_x_miss, y=shot_y_miss,
            mode='markers',
            marker=dict(size=10, color='black', symbol='circle', opacity=0.7),
            name='Shots'
        ))

    if shot_x_goal and shot_y_goal:
        fig.add_trace(go.Scatter(
            x=shot_x_goal, y=shot_y_goal,
            mode='markers',
            marker=dict(size=12, color='red', symbol='star', opacity=0.9),
            name='Goals'
        ))

    fig = draw_pitch_vertical(fig)

    fig.update_layout(
        
        xaxis=dict(range=[80, 0], visible=False),
        yaxis=dict(range=[120, 0], visible=False, scaleanchor='x'),
        plot_bgcolor='white',
        height=800,
        margin=dict(l=20, r=20, t=40, b=20)
    )

    return fig

# -------------------- DASH APP -------------------- #

def create_dash_app(engine):
    app = dash.Dash(__name__)
    app.title = "Football Heatmap Dashboard"
    player_options = load_players(engine)

    app.layout = html.Div([
        # Header
        html.Div([
            html.Div([
                html.H1("Football Analytics App", style={'margin': 0}),
                
            ], style={'flex': 1}),
            html.Div([
                html.A("GitHub", href="#", className="header-link"), 
                html.A("Dashboard", href="#", className="header-link")
               
            ], style={'display': 'flex', 'gap': '20px'})
        ], className="app-header"),

        # Main body
        html.Div([
            # Sidebar
            html.Div([
                html.H3("Player Selection", className="sidebar-title"),
                dcc.Dropdown(
                    id='player-dropdown',
                    options=player_options,
                    value=player_options[0]['value'] if player_options else None,
                    className="player-select"
                ),
                html.H3("Match Filter", className="sidebar-title"),
                dcc.Checklist(
                    id='match-checklist',
                    options=[],
                    value=[],
                    className="match-checklist"
                ),
                html.Div([
                    html.H3("Dashboard Guide", className="sidebar-title"),
                    html.Ul([
                        html.Li("Select a player to view their heatmap"),
                        html.Li("Check matches to filter by specific games"),
                        html.Li("Red stars indicate goals scored")
                    ])
                ], className="info-box")
            ], className="sidebar"),

            # Graph
            html.Div([
                dcc.Loading(
                    id='loading-graph',
                    type='circle',
                    children=dcc.Graph(id='heatmap-graph',config={'displayModeBar': True,'editable': False,'staticPlot': False,'modeBarButtonsToRemove': ['select', 'lasso2d'],'displaylogo': False})
                )
            ], style={'flex': 1, 'padding': '20px'})
        ], style={'display': 'flex', 'gap': '20px', 'padding': '20px'})
    ])

   
    @app.callback(
        [Output('match-checklist', 'options'), Output('match-checklist', 'value')],
        Input('player-dropdown', 'value')
    )
    def update_match_options(player_id):
        if player_id is None:
            return [], []
        options = load_player_matches(player_id, engine)
        values = [opt['value'] for opt in options]
        return options, values

    @app.callback(
        Output('heatmap-graph', 'figure'),
        Input('player-dropdown', 'value'),
        Input('match-checklist', 'value')
    )
    def update_heatmap(player_id, selected_matches):
        if not player_id:
            return go.Figure().update_layout(title="Please select a player.")

        if not selected_matches:
            fig = draw_pitch_vertical(go.Figure())
            fig.update_layout(
                title='Please select a match.',
                xaxis=dict(range=[80, 0], visible=False),
                yaxis=dict(range=[120, 0], visible=False, scaleanchor='x'),
                plot_bgcolor='white',
                height=800,
                margin=dict(l=20, r=20, t=40, b=20)
            )
            return fig

        kde_data = compute_kde(player_id, selected_matches)
        return create_vertical_heatmap(kde_data)

    return app

# -------------------- MAIN -------------------- #

if __name__ == '__main__':
    app = create_dash_app(engine)
    app.run(debug=True)
